In [ ]:
import numpy as np

from pipefunc import Pipeline, pipefunc

# -----------------------------------------------------------------------------
# Initial Conditions and Stage 1 Calculations
# -----------------------------------------------------------------------------


@pipefunc(output_name=("initial_altitude", "initial_velocity", "initial_time"))
def set_initial_conditions() -> tuple[float, float, float]:
    """Sets the initial conditions."""
    return 0.0, 0.0, 0.0


@pipefunc(output_name="stage_mass")
def calculate_stage_mass(stage1_mass_dry: float, stage1_mass_fuel: float) -> float:
    """Calculates the total mass of stage 1."""
    stage_mass = stage1_mass_dry + stage1_mass_fuel
    print(f"Stage 1 Total Mass: {stage_mass:.2f} kg")
    return stage_mass


@pipefunc(output_name="stage1_thrust_sl")
def calculate_thrust_stage1_sl(
    stage1_thrust_sl_solid_rocket: float,
    stage1_engine_type: str,
) -> float:
    """Calculates stage 1 thrust at sea level."""
    thrust = stage1_thrust_sl_solid_rocket
    print(f"Stage 1 Engine Type (Sea-Level): {stage1_engine_type}")
    print(f"Stage 1 Thrust (Sea-Level): {thrust:.2e} N")
    return thrust


@pipefunc(output_name="stage1_thrust_vac")
def calculate_thrust_stage1_vac(
    stage1_thrust_vac_solid_rocket: float,
    stage1_engine_type: str,
) -> float:
    """Calculates stage 1 thrust in a vacuum."""
    thrust = stage1_thrust_vac_solid_rocket
    print(f"Stage 1 Engine Type (Vacuum): {stage1_engine_type}")
    print(f"Stage 1 Thrust (Vacuum): {thrust:.2e} N")
    return thrust


@pipefunc(output_name="gravity_force")
def calculate_gravity_force(stage_mass: float) -> float:
    """Calculates gravity force."""
    gravity_acceleration = 9.81
    gravity = stage_mass * gravity_acceleration
    print(f"Gravity Force: {gravity:.2e} N")
    return gravity


@pipefunc(output_name="rocket_frontal_area")
def calculate_rocket_frontal_area(rocket_diameter: float) -> float:
    """Calculates the rocket's frontal area (simplified circle)."""
    radius = rocket_diameter / 2.0
    frontal_area = np.pi * radius**2
    print(f"Rocket Frontal Area: {frontal_area:.2f} m^2")
    return frontal_area


@pipefunc(output_name="aerodynamic_drag_stage1")
def calculate_aerodynamic_drag_stage1(
    initial_velocity: float,
    rocket_frontal_area: float,
    drag_coefficient: float,
    rocket_length: float,
) -> float:
    """Calculates aerodynamic drag force during stage 1 (very simplified)."""
    air_density = 1.225  # kg/m^3 (sea level, approx)
    drag_force = 0.5 * air_density * initial_velocity**2 * drag_coefficient * rocket_frontal_area
    print(f"Rocket Length (for context): {rocket_length:.2f} m")
    print(f"Aerodynamic Drag (Stage 1): {drag_force:.2e} N")
    return drag_force


@pipefunc(output_name="net_force_stage1")
def calculate_net_force_stage1(
    stage1_thrust_sl: float,
    gravity_force: float,
    aerodynamic_drag_stage1: float,
) -> float:
    """Calculates net force with drag considered."""
    net_force = stage1_thrust_sl - gravity_force - aerodynamic_drag_stage1  # Now subtracting drag
    print(f"Net Force (Stage 1, with Drag): {net_force:.2e} N")
    return net_force


@pipefunc(output_name="acceleration_stage1")
def calculate_acceleration_stage1(net_force_stage1: float, stage_mass: float) -> float:
    """Calculates acceleration with drag."""
    acceleration = net_force_stage1 / stage_mass
    print(f"Acceleration (Stage 1, with Drag): {acceleration:.2f} m/s^2")
    return acceleration


@pipefunc(output_name="velocity_stage1")
def update_velocity_stage1(acceleration_stage1: float, initial_velocity: float) -> float:
    """Updates velocity with drag."""
    time_step = 1.0
    updated_velocity = initial_velocity + acceleration_stage1 * time_step
    print(f"Velocity (Stage 1, with Drag): {updated_velocity:.2f} m/s")
    return updated_velocity


@pipefunc(output_name="altitude_stage1")
def update_altitude_stage1(velocity_stage1: float, initial_altitude: float) -> float:
    """Updates altitude with drag."""
    time_step = 1.0
    updated_altitude = initial_altitude + velocity_stage1 * time_step
    print(f"Altitude (Stage 1, with Drag): {updated_altitude:.2f} m")
    return updated_altitude


# -----------------------------------------------------------------------------
# Stage 2 Calculations
# -----------------------------------------------------------------------------


@pipefunc(output_name="stage2_total_mass")
def calculate_stage2_total_mass(stage2_mass_dry: float, stage2_mass_fuel: float) -> float:
    """Calculates the total mass of stage 2."""
    stage2_total_mass = stage2_mass_dry + stage2_mass_fuel
    print(f"Stage 2 Total Mass: {stage2_total_mass:.2f} kg")
    return stage2_total_mass


@pipefunc(output_name="stage2_thrust")
def calculate_thrust_stage2(stage2_thrust_vac: float, stage2_engine_type: str) -> float:
    """Calculates stage 2 thrust, using stage2_thrust_vac and stage2_engine_type."""
    thrust = stage2_thrust_vac
    print(f"Stage 2 Engine Type (Vacuum): {stage2_engine_type}")
    print(f"Stage 2 Thrust (Vacuum): {thrust:.2e} N")
    return thrust


@pipefunc(output_name="net_force_stage2")
def calculate_net_force_stage2(stage2_thrust: float, stage2_total_mass: float) -> float:
    """Calculates net force for stage 2."""
    gravity_force = stage2_total_mass * 9.81
    net_force = stage2_thrust - gravity_force
    print(f"Net Force (Stage 2): {net_force:.2e} N")
    return net_force


@pipefunc(output_name="acceleration_stage2")
def calculate_acceleration_stage2(net_force_stage2: float, stage2_total_mass: float) -> float:
    """Calculates acceleration for stage 2."""
    acceleration = net_force_stage2 / stage2_total_mass
    print(f"Acceleration (Stage 2): {acceleration:.2f} m/s^2")
    return acceleration


@pipefunc(output_name="velocity_stage2")
def update_velocity_stage2(acceleration_stage2: float, velocity_stage1: float) -> float:
    """Updates velocity for stage 2."""
    time_step = 1.0
    updated_velocity = velocity_stage1 + acceleration_stage2 * time_step
    print(f"Velocity (Stage 2): {updated_velocity:.2f} m/s")
    return updated_velocity


@pipefunc(output_name="altitude_stage2")
def update_altitude_stage2(velocity_stage2: float, altitude_stage1: float) -> float:
    """Updates altitude for stage 2."""
    time_step = 1.0
    updated_altitude = altitude_stage1 + velocity_stage2 * time_step
    print(f"Altitude (Stage 2): {updated_altitude:.2f} m")
    return updated_altitude


# -----------------------------------------------------------------------------
# Engine Performance Analysis
# -----------------------------------------------------------------------------


@pipefunc(output_name="stage1_engine_performance")
def analyze_stage1_engine_performance(
    stage1_engine_type: str,
    stage1_isp_sl: float,
    stage1_isp_vac: float,
) -> str:
    """Analyzes stage 1 engine performance."""
    performance_report = "Stage 1 Engine Performance Analysis:\n"
    performance_report += f"  Engine Type: {stage1_engine_type}\n"
    performance_report += f"  Specific Impulse (Sea Level): {stage1_isp_sl:.2f} s\n"
    performance_report += f"  Specific Impulse (Vacuum): {stage1_isp_vac:.2f} s"
    print(performance_report)
    return performance_report


@pipefunc(output_name="stage2_engine_performance")
def analyze_stage2_engine_performance(stage2_engine_type: str, stage2_isp_vac: float) -> str:
    """Analyzes stage 2 engine performance."""
    performance_report = "Stage 2 Engine Performance Analysis:\n"
    performance_report += f"  Engine Type: {stage2_engine_type}\n"
    performance_report += f"  Specific Impulse (Vacuum): {stage2_isp_vac:.2f} s"
    print(performance_report)
    return performance_report


# -----------------------------------------------------------------------------
# Pipeline Construction
# -----------------------------------------------------------------------------

rocket_pipeline = Pipeline(
    [
        # Stage 1 Calculations
        set_initial_conditions,  # Sets initial conditions
        calculate_stage_mass,  # Calculates stage 1 total mass
        calculate_thrust_stage1_sl,  # Calculates stage 1 thrust at sea level
        calculate_thrust_stage1_vac,  # Calculates stage 1 thrust in a vacuum
        calculate_gravity_force,  # Calculates gravity force
        calculate_rocket_frontal_area,  # Calculates rocket frontal area
        calculate_aerodynamic_drag_stage1,  # Calculates aero drag
        calculate_net_force_stage1,  # Net force considering drag
        calculate_acceleration_stage1,  # Calculates acceleration with drag
        update_velocity_stage1,  # Updates velocity with drag
        update_altitude_stage1,  # Updates altitude with drag
        # Stage 2 Calculations
        calculate_stage2_total_mass,  # Calculates stage 2 total mass
        calculate_thrust_stage2,  # Calculates stage 2 thrust
        calculate_net_force_stage2,  # Calculates net force for stage 2
        calculate_acceleration_stage2,  # Calculates acceleration for stage 2
        update_velocity_stage2,  # Updates velocity for stage 2
        update_altitude_stage2,  # Updates altitude for stage 2
        # Engine Performance Analysis
        analyze_stage1_engine_performance,  # Analyzes stage 1 engine performance
        analyze_stage2_engine_performance,  # Analyzes stage 2 engine performance
    ],
)


inputs = {
    "stage1_engine_type": "SolidRocketMotor",
    "stage2_engine_type": "VacuumOptimizedEngine",
    "stage1_mass_dry": 1000.0,
    "stage1_mass_fuel": 5000.0,
    "stage2_mass_dry": 500.0,
    "stage2_mass_fuel": 2000.0,
    "stage1_thrust_sl_solid_rocket": 1e6,
    "stage1_thrust_vac_solid_rocket": 1.2e6,
    "stage1_isp_sl": 250.0,
    "stage1_isp_vac": 280.0,
    "stage2_thrust_vac": 2e5,
    "stage2_isp_vac": 350.0,
    "rocket_diameter": 4.0,
    "rocket_length": 30.0,
    "drag_coefficient": 0.5,
}

rocket_pipeline.map(inputs)
rocket_pipeline.visualize(backend="graphviz")

In [ ]:
rocket_pipeline.leaf_nodes